## git_SKT_기상데이터_상관성분석 (2021년 기준)

- 날짜 : 22.12.27

In [1]:
import os
import sys
sys.path.append(os.pardir)

import numpy as np
import pandas as pd
from utils import *

동
--
동부소방서, 성산하수처리장, 수산1리 복지회관, 난산리 다목적회관, 김영갑갤러리, 종달리 종합복지회관, 구좌119센터, 제주해녀박물관, 테크노파크 용암해수센터, 스마트그리드 홍보관

서
--
영어교육도시119센터, 유리의성, 생각하는정원, 저지리사무소, 김창열미술관, 서부농업기술센터, 동광리버스차고지, 서커스월드공연장, 토이파크


남
--
서귀포혁신LH1단지, 서귀포혁신LH2단지, 서귀포해양경찰서, 서귀포지사, 강정상록 아파트, 강창학경기장, 농업기술원, 대천동주민센터, 강정생명평화교회, 제주서귀포동홍6단지LH 아파트, 천지공영주차장, 제주권역재활센터, 서귀포동홍3LH 아파트, 서귀포향토오일장, 정방폭포


북
--
제주직할, 제주도교육청, 제주특별자치도 복지이음마루, 행복날개 주유소, 공항입구 공영주차장, 제주공항, 연동한일시티, 연동대림1차아파트, 해오름 아파트, 노형2차부영 아파트, 제주으뜸마을LH 아파트, 제주부영1차 아파트, 노형 제2공영주차장, 제주정든마을1단지LH 아파트, 제주시민속오일시장 3주차장, 한라산렌트카, 개인택시조합, 도두동 주민센터

## 기상데이터 Load : 2021년 동서남북

In [2]:
# 기상데이터 Load : 2021년 동서남북

filepath = 'data'
filename_east = "기상방재_제주_송당862_202101_202112"
filename_west = "기상방재_제주_서광752_202101_202112"
filename_south = "기상방재_제주_강정980_202101_202112"
filename_north = "기상방재_제주_외도863_202101_202112"
print("[*] Load the dataset")

filename_set = [filename_east, filename_west, filename_south, filename_north]
weather_set = []
for filename in filename_set:
    raw_df = pd.read_csv(os.path.join(filepath, "%s.csv" % filename),
                     engine='python', encoding='cp949',
                     index_col='일시', parse_dates=True)       # index_col='수정 가능' !
    raw_df.sort_index(inplace=True)
    
    raw_df.columns = [
        'loc', 'locNm', 'temp', 'winddirection(deg)', 'windspeed(m/s)', 'rainfall(mm)',
        'pressure(hPa)', 'pressure2(hPa)', 'humidity(%)'
    ]
    raw_df.index.name='datetime'
    
    # 현지기압, 해면기압은 1년동안 아예 데이터가 없기도 해서 삭제
    raw_df = remove_df_col(raw_df, ['pressure(hPa)', 'pressure2(hPa)'])
    
    n_nulls = raw_df.isnull().sum().sum()
    print(f"Null 값의 개수: {n_nulls}")
    
    weather_set.append(raw_df)

[*] Load the dataset
Null 값의 개수: 106
Null 값의 개수: 166
Null 값의 개수: 39
Null 값의 개수: 126


## 기상데이터 결측치 보완

In [3]:
# 기상데이터 결측치 보완

coln_set = ['temp', 'winddirection(deg)', 'windspeed(m/s)', 'rainfall(mm)', 'humidity(%)']

weather_set_final = []
for weather in weather_set:
    w = null_add_general_info(weather)
    
    for coln in coln_set:
        w = replace_null_ymhavg(w, coln)
        
    w = remove_df_col(w, ['year', 'month', 'day', 'week', 'hour', 'minute'])
    
    n_nulls = w.isnull().sum().sum()
    print(f"Null 값의 개수: {n_nulls}")
    assert n_nulls == 0, "NaN 값 존재"
        
    weather_set_final.append(w)
    
weather_east, weather_west, weather_south, weather_north = (
    weather_set_final[0], weather_set_final[1], weather_set_final[2], weather_set_final[3]
)

[*] Null 값 대체
temp의 Null 값 개수: 2
temp의 Null 값 날짜 : DatetimeIndex(['2021-04-15 00:00:00', '2021-11-03 03:00:00'], dtype='datetime64[ns]', name='datetime', freq=None)
[*] Null 값 대체
winddirection(deg)의 Null 값 개수: 25
winddirection(deg)의 Null 값 날짜 : DatetimeIndex(['2021-04-15 00:00:00', '2021-06-02 12:00:00',
               '2021-06-02 13:00:00', '2021-06-02 14:00:00',
               '2021-06-13 04:00:00', '2021-06-13 05:00:00',
               '2021-06-13 06:00:00', '2021-08-02 07:00:00',
               '2021-08-08 06:00:00', '2021-08-08 07:00:00',
               '2021-08-12 03:00:00', '2021-08-12 04:00:00',
               '2021-08-23 04:00:00', '2021-11-03 03:00:00',
               '2021-11-19 04:00:00', '2021-11-19 05:00:00',
               '2021-11-19 06:00:00', '2021-11-21 01:00:00',
               '2021-11-21 02:00:00', '2021-11-21 03:00:00',
               '2021-11-28 06:00:00', '2021-11-28 07:00:00',
               '2021-11-28 08:00:00', '2021-11-28 09:00:00',
               '2021-12

[*] Null 값 대체
winddirection(deg)의 Null 값 개수: 22
winddirection(deg)의 Null 값 날짜 : DatetimeIndex(['2021-04-16 07:00:00', '2021-04-16 19:00:00',
               '2021-04-17 11:00:00', '2021-04-17 15:00:00',
               '2021-04-17 18:00:00', '2021-04-17 23:00:00',
               '2021-04-18 00:00:00', '2021-04-18 22:00:00',
               '2021-04-21 01:00:00', '2021-04-21 02:00:00',
               '2021-04-21 03:00:00', '2021-04-21 04:00:00',
               '2021-04-21 05:00:00', '2021-04-21 06:00:00',
               '2021-04-21 07:00:00', '2021-04-21 08:00:00',
               '2021-04-21 09:00:00', '2021-07-12 15:00:00',
               '2021-07-12 16:00:00', '2021-07-12 17:00:00',
               '2021-07-30 09:00:00', '2021-08-02 07:00:00'],
              dtype='datetime64[ns]', name='datetime', freq=None)
[*] Null 값 대체
windspeed(m/s)의 Null 값 개수: 22
windspeed(m/s)의 Null 값 날짜 : DatetimeIndex(['2021-04-16 07:00:00', '2021-04-16 19:00:00',
               '2021-04-17 11:00:00', '2021-04-17

## 충전소 시계열 Load : 2021년

In [4]:
# 충전소 시계열 Load : 2021년

filepath = 'data'
filename = "SKT_최종개별충전소_시계열_22.12.12"
print("[*] Load the dataset")

raw_df = pd.read_csv(os.path.join(filepath, "%s.csv" % filename),
                     engine='python', encoding='cp949',
                     index_col='datetime', parse_dates=True)       # index_col='수정 가능' !
raw_df.sort_index(inplace=True)

n_nulls = raw_df.isnull().sum().sum()
print(f"Null 값의 개수: {n_nulls}")
assert n_nulls == 0, "NaN 값 존재"

print("raw_df length :", len(raw_df))

ev_df = raw_df
ev_df = ev_df.loc['2021-01-01':'2021-12-31', :]

[*] Load the dataset
Null 값의 개수: 0
raw_df length : 21888


## 충전소 시계열의 합 : 동서남북

In [5]:
# 동쪽 충전소
statn_east = [
    '동부소방서', '성산하수처리장', '수산1리 복지회관', '난산리 다목적회관', '김영갑갤러리', '종달리 종합복지회관',
    '구좌119센터', '제주해녀박물관', '테크노파크 용암해수센터', '스마트그리드 홍보관',
]

# 서쪽 충전소
statn_west = [
    '영어교육도시119센터', '유리의성', '생각하는정원', '저지리사무소', '김창열미술관', '서부농업기술센터',
    '동광리버스차고지', '서커스월드공연장', '토이파크',
]

# 남쪽 충전소
statn_south = [
    '서귀포혁신LH1단지', '서귀포혁신LH2단지', '서귀포해양경찰서', '서귀포지사', '강정상록 아파트',
    '강창학경기장', '농업기술원', '대천동주민센터', '강정생명평화교회', '제주서귀포동홍6단지LH 아파트',
    '천지공영주차장', '제주권역재활센터', '서귀포동홍3LH 아파트', '서귀포향토오일장', '정방폭포',
]

# 북쪽 충전소
statn_north = [
    '제주직할', '제주도교육청', '제주특별자치도 복지이음마루', '행복날개 주유소', '공항입구 공영주차장',
    '제주공항', '연동한일시티', '연동대림1차아파트', '해오름 아파트', '노형2차부영 아파트',
    '제주으뜸마을LH 아파트', '제주부영1차 아파트', '노형 제2공영주차장', '제주정든마을1단지LH 아파트', 
    '제주시민속오일시장 3주차장', '한라산렌트카', '개인택시조합', '도두동 주민센터',
]

# 동쪽 충전소 시계열 합
stat_east = ev_df.copy()[statn_east[0]]  # first
for s in statn_east[1:]:
    stat_east += ev_df[s]
stat_east = pd.DataFrame(stat_east)
stat_east.columns = ['ev_east']

# 서쪽 충전소 시계열 합
stat_west = ev_df.copy()[statn_west[0]]  # first
for s in statn_west[1:]:
    stat_west += ev_df[s]
stat_west = pd.DataFrame(stat_west)
stat_west.columns = ['ev_west']

# 남쪽 충전소 시계열 합
stat_south = ev_df.copy()[statn_south[0]]  # first
for s in statn_south[1:]:
    stat_south += ev_df[s]
stat_south = pd.DataFrame(stat_south)
stat_south.columns = ['ev_south']

# 북쪽 충전소 시계열 합
stat_north = ev_df.copy()[statn_north[0]]  # first
for s in statn_north[1:]:
    stat_north += ev_df[s]
stat_north = pd.DataFrame(stat_north)
stat_north.columns = ['ev_north']

## 상관성 분석 (피어슨 상관계수)

In [6]:
# 기상데이터만 남김
weather_east = remove_df_col(weather_east, ['loc', 'locNm'])
weather_west = remove_df_col(weather_west, ['loc', 'locNm'])
weather_south = remove_df_col(weather_south, ['loc', 'locNm'])
weather_north = remove_df_col(weather_north, ['loc', 'locNm'])

# 기상데이터 및 EV 데이터 통합
d_east = pd.concat([weather_east, stat_east], axis=1)
d_west = pd.concat([weather_west, stat_west], axis=1)
d_south = pd.concat([weather_south, stat_south], axis=1)
d_north = pd.concat([weather_north, stat_north], axis=1)

# 상관성 분석 (피어슨 상관계수)
corr_mat_east, corr_target_east = corr_model(d_east, 'ev_east', 'pearson')
corr_mat_west, corr_target_west = corr_model(d_west, 'ev_west', 'pearson')
corr_mat_south, corr_target_south = corr_model(d_south, 'ev_south', 'pearson')
corr_mat_north, corr_target_north = corr_model(d_north, 'ev_north', 'pearson')

In [7]:
display(corr_target_east)
display(corr_target_west)
display(corr_target_south)
display(corr_target_north)

,ev_east
ev_east,1.000000
windspeed(m/s),0.185782
winddirection(deg),0.055286
temp,0.027414
rainfall(mm),-0.035366
humidity(%),-0.233906


,ev_west
ev_west,1.000000
winddirection(deg),0.151151
temp,0.122808
windspeed(m/s),0.096202
rainfall(mm),-0.043217
humidity(%),-0.247296


,ev_south
ev_south,1.000000
windspeed(m/s),0.145299
winddirection(deg),0.132226
temp,0.108565
rainfall(mm),-0.047938
humidity(%),-0.287585


,ev_north
ev_north,1.000000
temp,0.149850
windspeed(m/s),0.147882
winddirection(deg),0.018288
rainfall(mm),-0.037711
humidity(%),-0.258276
